# Building Deep-Network Structures in pyhgf

This document demonstrates two new high-level functions for constructing layered, fully connected value-parent structures in `pyhgf`:

- **`add_layer()`** – adds a *single* fully connected parent layer.
- **`add_value_parent_stack()`** – builds *multiple* layers at once, similar to `Sequential` in deep-learning frameworks.

These functions allow HGF models to be composed in a deep-network style while remaining fully compatible with the probabilistic belief-update dynamics.

In [1]:
from pyhgf.model import Network

## Creating the Base Layer

We begin by defining a small bottom (input) layer of 4 continuous nodes:

In [2]:
net = Network().add_nodes(kind="continuous-state", n_nodes=4, precision=5.0)
bottom_idxs = list(range(4))

## Adding One Layer with `add_layer`

`add_layer` provides fine-grained control, letting you manually construct each layer.

This is useful when each layer should have different hyperparameters (precision, tonic volatility, autoconnection strength, etc.). The function creates a fully connected parent layer, where all new parent nodes connect to all children below them.

By default, `add_layer` automatically connects to all orphan nodes (nodes without value parents). You can also specify `value_children` explicitly to control which nodes the layer connects to.

In [ ]:
# Add a single parent layer of 3 nodes
# Note: Method chaining - add_layer returns the network
net = net.add_layer(
    size=3,
    value_children=bottom_idxs,
    precision=1.0,
    tonic_volatility=-1.0,
    autoconnection_strength=0.1,
)

# Now add a second parent layer of 2 nodes
# Since the previous layer has no parents, it will automatically connect to it
net = net.add_layer(
    size=2,
    precision=0.5,
    tonic_volatility=-2.0,
    autoconnection_strength=1.0,
)

# Or chain them in a single expression (like Keras/PyTorch):
net = (
    Network()
    .add_nodes(kind="continuous-state", n_nodes=4, precision=5.0)
    .add_layer(size=3, precision=1.0, tonic_volatility=-1.0)
    .add_layer(size=2, precision=0.5, tonic_volatility=-2.0)
)

# Visualize the network structure
net.plot_network()

In [ ]:
# If you need to track layer indices for visualization, you can compute them manually:
bottom_layer = list(range(4))  # Nodes 0-3
layer1 = list(range(4, 7))     # Nodes 4-6 (3 nodes)
layer2 = list(range(7, 9))     # Nodes 7-8 (2 nodes)

layers = [bottom_layer, layer1, layer2]
net.plot_deep_network(layers)

## Adding Multiple Layers with `add_value_parent_stack`

`add_value_parent_stack` provides a compact way to build several fully connected parent layers at once. Instead of adding each layer manually, you simply specify the desired layer sizes (e.g., [3, 16, 32]), and the function creates them sequentially. Each layer is fully connected to the one below, using the same hyperparameters for all layers you add (precision, tonic volatility, autoconnection strength, etc.).

This is ideal when you want to quickly prototype deep hierarchical networks or mimic the "stacked layer" construction found in deep learning frameworks.

Like `add_layer`, it also supports method chaining and auto-connects to orphan nodes by default.

In [ ]:
# Add 3 fully connected parent layers (4→4→16→32) using method chaining
net = (
    Network()
    .add_nodes(kind="continuous-state", n_nodes=4, precision=5.0)
    .add_value_parent_stack(
        layer_sizes=[4, 16, 32],
        precision=1.0,
        tonic_volatility=-1.0,
        autoconnection_strength=0.2,
    )
)

# For visualization, manually compute layer indices
layers = [
    list(range(0, 4)),      # Bottom: nodes 0-3
    list(range(4, 8)),      # Layer 1: nodes 4-7 (4 nodes)
    list(range(8, 24)),     # Layer 2: nodes 8-23 (16 nodes)
    list(range(24, 56)),    # Layer 3: nodes 24-55 (32 nodes)
]

net.plot_deep_network(layers)